In [8]:
# Initial imports
#import os
import requests
import pandas as pd
from yahoofinancials import YahooFinancials
import numpy as np
#from dotenv import load_dotenv
#import alpaca_trade_api as tradeapi
#from MCForecastTools import MCSimulation

%matplotlib inline

In [9]:
# using Yahoo Finance API
tech_stocks = ['AAPL','TSLA', 'DDD']
#bank_stocks = ['WFC', 'BAC', 'C']
commodity_futures = ['GC=F', 'SI=F', 'CL=F']
cryptocurrencies = ['BTC-USD', 'ETH-USD', 'XRP-USD']
#currencies = ['EURUSD=X', 'JPY=X', 'GBPUSD=X']
#us_treasuries = ['^TNX', '^IRX', '^TYX']

#yahoo_financials_tech = YahooFinancials(tech_stocks)
#yahoo_financials_banks = YahooFinancials(bank_stocks)
yahoo_financials_commodities = YahooFinancials(commodity_futures)
yahoo_financials_cryptocurrencies = YahooFinancials(cryptocurrencies)
#yahoo_financials_currencies = YahooFinancials(currencies)
#yahoo_financials_mutualfunds = YahooFinancials(mutual_funds)
#yahoo_financials_treasuries = YahooFinancials(us_treasuries)
#tech_cash_flow_data_an = yahoo_financials_tech.get_financial_stmts('annual', 'cash')
#tech_bal_data = yahoo_financials_tech.get_financial_stmts('annual', 'balance')
#tech_income_data = yahoo_financials_tech.get_financial_stmts('annual', 'income')
#bank_cash_flow_data_an = yahoo_financials_banks.get_financial_stmts('annual', 'cash')

#banks_net_ebit = yahoo_financials_banks.get_ebit()
#tech_stock_price_data = yahoo_financials_tech.get_stock_price_data()

#daily_bank_stock_prices = yahoo_financials_banks.get_historical_price_data('2008-09-15', '2018-09-15', 'daily')
#daily_commodity_prices = yahoo_financials_commodities.get_historical_price_data('2008-09-15', '2018-09-15', 'daily')
daily_crypto_prices = yahoo_financials_cryptocurrencies.get_historical_price_data('2008-09-15', '2018-09-15', 'daily')
#daily_currency_prices = yahoo_financials_currencies.get_historical_price_data('2008-09-15', '2018-09-15', 'daily')
#daily_mutualfund_prices = yahoo_financials_mutualfunds.get_historical_price_data('2008-09-15', '2018-09-15', 'daily')
#daily_treasury_prices = yahoo_financials_treasuries.get_historical_price_data('2008-09-15', '2018-09-15', 'daily')

In [16]:
display(yahoo_financials_cryptocurrencies.get_50day_moving_avg())
yahoo_financials_cryptocurrencies.get_summary_data()


{'BTC-USD': 17286.879, 'ETH-USD': 1271.7393, 'XRP-USD': 0.36799997}

{'BTC-USD': {'previousClose': 21179.898,
  'regularMarketOpen': 21179.898,
  'twoHundredDayAverage': 19534.592,
  'trailingAnnualDividendYield': None,
  'payoutRatio': None,
  'volume24Hr': 23936028672,
  'regularMarketDayHigh': 21258.217,
  'navPrice': None,
  'averageDailyVolume10Day': 21954400932,
  'totalAssets': None,
  'regularMarketPreviousClose': 21179.898,
  'fiftyDayAverage': 17286.879,
  'trailingAnnualDividendRate': None,
  'open': 21179.898,
  'toCurrency': 'USD=X',
  'averageVolume10days': 21954400932,
  'expireDate': '-',
  'yield': None,
  'algorithm': None,
  'dividendRate': None,
  'exDividendDate': '-',
  'beta': None,
  'circulatingSupply': 19265400,
  'startDate': '2013-04-28',
  'regularMarketDayLow': 21173.096,
  'priceHint': 2,
  'currency': 'USD',
  'regularMarketVolume': 23936028672,
  'lastMarket': 'CoinMarketCap',
  'maxSupply': None,
  'openInterest': None,
  'marketCap': 409240043520,
  'volumeAllCurrencies': 23936028672,
  'strikePrice': None,
  'averageV

In [10]:
daily_crypto_prices

{'BTC-USD': {'eventsData': {},
  'firstTradeDate': {'formatted_date': '2014-09-17', 'date': 1410912000},
  'currency': 'USD',
  'instrumentType': 'CRYPTOCURRENCY',
  'timeZone': {'gmtOffset': 0},
  'prices': [{'date': 1410912000,
    'high': 468.17401123046875,
    'low': 452.4219970703125,
    'open': 465.864013671875,
    'close': 457.3340148925781,
    'volume': 21056800,
    'adjclose': 457.3340148925781,
    'formatted_date': '2014-09-17'},
   {'date': 1410998400,
    'high': 456.8599853515625,
    'low': 413.10400390625,
    'open': 456.8599853515625,
    'close': 424.44000244140625,
    'volume': 34483200,
    'adjclose': 424.44000244140625,
    'formatted_date': '2014-09-18'},
   {'date': 1411084800,
    'high': 427.8349914550781,
    'low': 384.5320129394531,
    'open': 424.1029968261719,
    'close': 394.7959899902344,
    'volume': 37919700,
    'adjclose': 394.7959899902344,
    'formatted_date': '2014-09-19'},
   {'date': 1411171200,
    'high': 423.2959899902344,
    'lo